## EECS 738 - Machine learning 

### Reinforcement learning game - homework 4  

In [1]:
import numpy as np
import random

Building the environment

In [2]:
MONSTER = "m"
PLAYER = "p"
REWARD = "r"
EMPTY = "*"
TREE = "t"
END = "e"

grid = [
            [PLAYER, EMPTY, MONSTER, MONSTER],
            [EMPTY, EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY, EMPTY],
            [EMPTY, TREE, EMPTY, EMPTY],
            [REWARD, TREE, EMPTY, EMPTY],
            [EMPTY, TREE, EMPTY, EMPTY],
            [EMPTY, TREE, EMPTY, MONSTER],
            [EMPTY, TREE, EMPTY, EMPTY],
            [MONSTER, TREE, EMPTY, END]
        ]



for row in grid:
    print(' '.join(row))

p * m m
* * * *
* * * *
* * * *
* t * *
r t * *
* t * *
* t * m
* t * *
m t * e


Wrap our environment state in a class that holds the current grid and player position

In [3]:
class State:
    
    def __init__(self, grid, player_pos):
        self.grid = grid
        self.player_pos = player_pos
        
    def __eq__(self, other):
        return isinstance(other, State) and self.grid == other.grid and self.player_pos == other.player_pos
    
    def __hash__(self):
        return hash(str(self.grid) + str(self.player_pos))
    
    def __str__(self):
        return f"State(grid={self.grid}, player_pos={self.player_pos})"

Actions and init state

In [4]:
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

ACTIONS = [UP, DOWN, LEFT, RIGHT]

start_state = State(grid=grid, player_pos=[0, 0])

Function that takes the current state with an action and returns new state, reward and whether or not the episode has completed


In [5]:
from copy import deepcopy

def act(state, action):
    
    def new_player_pos(state, action):
        p = deepcopy(state.player_pos)
        if action == UP:
            p[0] = max(0, p[0] - 1)
        elif action == DOWN:
            p[0] = min(len(state.grid) - 1, p[0] + 1)
        elif action == LEFT:
            p[1] = max(0, p[1] - 1)
        elif action == RIGHT:
            p[1] = min(len(state.grid[0]) - 1, p[1] + 1)
        else:
            raise ValueError(f"Unknown action {action}")
        return p
            
    p = new_player_pos(state, action)
    grid_item = state.grid[p[0]][p[1]]
    
    new_grid = deepcopy(state.grid)
    
    # Defining the rules and the point system:
    
    # move to a square with monster will end the game (failure)
    if grid_item == MONSTER:
        reward = -1000
        is_done = True
        new_grid[p[0]][p[1]] += PLAYER
    elif grid_item == REWARD:
        reward = 1000
        is_done = False
        old = state.player_pos
        new_grid[old[0]][old[1]] = EMPTY
        new_grid[p[0]][p[1]] = PLAYER
    elif grid_item == EMPTY:
        reward = -1
        is_done = False
        old = state.player_pos
        new_grid[old[0]][old[1]] = EMPTY
        new_grid[p[0]][p[1]] = PLAYER
    # move to the end square will end the game (success)
    elif grid_item == END:
        reward = 5000
        is_done = True
        new_grid[p[0]][p[1]] += PLAYER
    # player cannot move to square with a tree
    elif grid_item == TREE:
        reward = -20
        is_done = False
    elif grid_item == PLAYER:
        reward = -1
        is_done = False
    else:
        raise ValueError(f"Unknown grid item {grid_item}")
    
    return State(grid=new_grid, player_pos=p), reward, is_done

Learning 

In [6]:
import numpy as np
import random

random.seed(42) 

N_STATES = len(grid[0]) * len(grid)
N_EPISODES = 1000

MAX_EPISODE_STEPS = 100

MIN_ALPHA = 0.02

alphas = np.linspace(1.0, MIN_ALPHA, N_EPISODES)
gamma = 1.0
eps = 0.2

q_table = dict()

A helper function q that gives the Q value for a state-action pair or for all actions, given a state

In [7]:
def q(state, action=None):
    
    if state not in q_table:
        q_table[state] = np.zeros(len(ACTIONS))
        
    if action is None:
        return q_table[state]
    
    return q_table[state][action]

Act with random action with some small probability or the best action seen so far (using our q_table)

In [8]:
def choose_action(state):
    if random.uniform(0, 1) < eps:
        return random.choice(ACTIONS) 
    else:
        return np.argmax(q(state))

Training agent using the Q-learning algorithm

In [10]:
for e in range(N_EPISODES):
    
    state = start_state
    total_reward = 0
    alpha = alphas[e]
    
    for _ in range(MAX_EPISODE_STEPS):
        action = choose_action(state)
        next_state, reward, done = act(state, action)
        total_reward += reward
        
        q(state)[action] = q(state, action) + \
                alpha * (reward + gamma *  np.max(q(next_state)) - q(state, action))
        state = next_state
        if done:
            break
    print(f"Run {e + 1}: total reward -> {total_reward} points")

Run 1: total reward -> 5984 points
Run 2: total reward -> 5984 points
Run 3: total reward -> 5942 points
Run 4: total reward -> 5975 points
Run 5: total reward -> 5963 points
Run 6: total reward -> 5984 points
Run 7: total reward -> -53 points
Run 8: total reward -> 5984 points
Run 9: total reward -> 5965 points
Run 10: total reward -> -147 points
Run 11: total reward -> -19 points
Run 12: total reward -> 5981 points
Run 13: total reward -> 5984 points
Run 14: total reward -> 5984 points
Run 15: total reward -> 5986 points
Run 16: total reward -> 5964 points
Run 17: total reward -> 5902 points
Run 18: total reward -> 5900 points
Run 19: total reward -> 5983 points
Run 20: total reward -> -23 points
Run 21: total reward -> 5984 points
Run 22: total reward -> 5963 points
Run 23: total reward -> 5965 points
Run 24: total reward -> 5961 points
Run 25: total reward -> 5958 points
Run 26: total reward -> 5984 points
Run 27: total reward -> 5959 points
Run 28: total reward -> 5976 points
Run 

Run 234: total reward -> 5986 points
Run 235: total reward -> 5982 points
Run 236: total reward -> 5981 points
Run 237: total reward -> 5979 points
Run 238: total reward -> 5986 points
Run 239: total reward -> -63 points
Run 240: total reward -> 5983 points
Run 241: total reward -> 5962 points
Run 242: total reward -> -13 points
Run 243: total reward -> 5980 points
Run 244: total reward -> 5875 points
Run 245: total reward -> 5976 points
Run 246: total reward -> 5986 points
Run 247: total reward -> 5984 points
Run 248: total reward -> 5959 points
Run 249: total reward -> 5940 points
Run 250: total reward -> 5980 points
Run 251: total reward -> 5982 points
Run 252: total reward -> -189 points
Run 253: total reward -> -191 points
Run 254: total reward -> 5984 points
Run 255: total reward -> -35 points
Run 256: total reward -> 5984 points
Run 257: total reward -> 5976 points
Run 258: total reward -> 5984 points
Run 259: total reward -> 5984 points
Run 260: total reward -> 5963 points
Run 

Run 490: total reward -> 5986 points
Run 491: total reward -> -165 points
Run 492: total reward -> 5986 points
Run 493: total reward -> 5982 points
Run 494: total reward -> 5982 points
Run 495: total reward -> 5983 points
Run 496: total reward -> 5964 points
Run 497: total reward -> 5955 points
Run 498: total reward -> 5963 points
Run 499: total reward -> 5983 points
Run 500: total reward -> 5980 points
Run 501: total reward -> 5984 points
Run 502: total reward -> 5961 points
Run 503: total reward -> 5981 points
Run 504: total reward -> 5986 points
Run 505: total reward -> 5955 points
Run 506: total reward -> 5898 points
Run 507: total reward -> -12 points
Run 508: total reward -> 5981 points
Run 509: total reward -> 5962 points
Run 510: total reward -> 5984 points
Run 511: total reward -> 5985 points
Run 512: total reward -> 5961 points
Run 513: total reward -> 5901 points
Run 514: total reward -> -190 points
Run 515: total reward -> 5980 points
Run 516: total reward -> 5984 points
Ru

Run 734: total reward -> 5928 points
Run 735: total reward -> 5964 points
Run 736: total reward -> 5965 points
Run 737: total reward -> 5975 points
Run 738: total reward -> 5961 points
Run 739: total reward -> -1056 points
Run 740: total reward -> 5980 points
Run 741: total reward -> 5984 points
Run 742: total reward -> 5974 points
Run 743: total reward -> 5983 points
Run 744: total reward -> 5986 points
Run 745: total reward -> 5944 points
Run 746: total reward -> 5986 points
Run 747: total reward -> 5986 points
Run 748: total reward -> 5986 points
Run 749: total reward -> 5982 points
Run 750: total reward -> 5984 points
Run 751: total reward -> 5962 points
Run 752: total reward -> 5975 points
Run 753: total reward -> 5984 points
Run 754: total reward -> 5986 points
Run 755: total reward -> 5982 points
Run 756: total reward -> 5979 points
Run 757: total reward -> 5978 points
Run 758: total reward -> 5963 points
Run 759: total reward -> 5980 points
Run 760: total reward -> 5979 points


Run 979: total reward -> -151 points
Run 980: total reward -> 5984 points
Run 981: total reward -> -116 points
Run 982: total reward -> 5986 points
Run 983: total reward -> 5963 points
Run 984: total reward -> -22 points
Run 985: total reward -> 5984 points
Run 986: total reward -> -37 points
Run 987: total reward -> 5941 points
Run 988: total reward -> 5976 points
Run 989: total reward -> 5977 points
Run 990: total reward -> 5959 points
Run 991: total reward -> 5962 points
Run 992: total reward -> -111 points
Run 993: total reward -> 5983 points
Run 994: total reward -> 5981 points
Run 995: total reward -> 5977 points
Run 996: total reward -> -34 points
Run 997: total reward -> 5886 points
Run 998: total reward -> 5975 points
Run 999: total reward -> 5962 points
Run 1000: total reward -> 5983 points


The more we train the model, the more points we are able to collect